# Introduction
GOAL: To train a model to predict the wild fire risk of properties, using census data as input features and the proximity to fires as a target feature.



In [1]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


import load_GIS 
import load_census

# from load_GIS import LoadGIS 
# from load_census import LoadCensus


# Load Training and Target Features

## Load 2023 US Census Data

In [2]:
census_filepaths=[]
census_df = load_census.LoadCensus(census_filepaths)

gis_filepaths=[]
gis_df = load_GIS.LoadGIS(gis_filepaths)

TypeError: LoadCensus.__init__() missing 1 required positional argument: 'filepaths'

## Select Properties of Interest

Chosen randomly from US addresses. We will join these to the relevant census data


## Load Wildfire GIS Data for 2023

We will use point data from the Visible Infrared Imaging Radiometer Suite (VIIRS). A valid alternative is using burn boundary data. There are a few different data sources we could use, but in the interest of (portfolio) simplicity we'll use just the VIIRS.



## Give Each Property a Wildfire Risk Score

Will be based on the proximity to wildfire points, weighted by the number of nearby fires, with a cutoff of 50km.

# Prepare Features for ML Training

## Train/Test Split

## Preliminary Training

Will do an initial training of a variety of ML types, and select the best one for a more refined hyperparameter search

## Refine Model Hyperparameters

# Conclusion